Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

---

# Testing Lab

### General Info

Testing is one of the most important components of sustainable software development. It improves code quality, maintainability and lifetime, and saves developer time. In previous labs you have already come across this in the form of assert statements. There are a number of testing libraries to support Python development and they all have in common that they build on the assert statement. In addition they provide better information when errors are found and the facilitate automated testing of large projects.

We will use the doctest and the pytest libraries. If pytest is not installed you need to install it with pip. On your computers do this in the virtual environment where you previously installed jupyter

~~~
$ pip install pytest
$ jupyter notebook
~~~

Testing libraries are typically designed to be used on Python source files while they are not adapted to be used with Jupyter notebooks. To be able to work with this in this lab, we use cell magic commands (%%file) to save cells in ordinary files and then execute pytest on those files

In [64]:
import doctest
import pytest
# This is for jupyter to recognize changes in external files without restarting kernel
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Assignment 1: add time stamps

In [65]:
%%file timestamps.py
def sum_timestamps(l):
    """
    >>> sum_timestamps(['5:32', '4:48'])
    '10:20'
    >>> sum_timestamps(['03:10', '01:00'])
    '4:10'
    >>> sum_timestamps(['2:10', '1:59'])
    '4:09'
    >>> sum_timestamps(['15:32', '45:48'])
    '1:01:20'
    >>> sum_timestamps(['6:15:32', '2:45:48'])
    '9:01:20'
    >>> sum_timestamps(['6:35:32', '2:45:48', '40:10'])
    '10:01:30'
    """
    import datetime
 
    tmstps = datetime.timedelta()
    for i in l:
        n = i.count(':')
        if n == 1:
            (m, s) = i.split(':')
            t = datetime.timedelta(minutes=int(m), seconds=int(s))
            tmstps += t
        elif n == 2:
            (h, m, s) = i.split(':')
            t = datetime.timedelta(hours=int(h), minutes=int(m), seconds=int(s))
            tmstps += t
    (h,m,s) = str(tmstps).split(':')
    if int(h) == 0:
        tmstps = [str(int(m)),str(s)]
        tmstps = ':'.join(tmstps)
    else:
        tmstps = [str(h),str(m),str(s)]
        tmstps = ':'.join(tmstps)
    
    return(tmstps)


Overwriting timestamps.py


In [66]:
import timestamps
doctest.testmod(timestamps, verbose=True)

Trying:
    sum_timestamps(['5:32', '4:48'])
Expecting:
    '10:20'
ok
Trying:
    sum_timestamps(['03:10', '01:00'])
Expecting:
    '4:10'
ok
Trying:
    sum_timestamps(['2:10', '1:59'])
Expecting:
    '4:09'
ok
Trying:
    sum_timestamps(['15:32', '45:48'])
Expecting:
    '1:01:20'
ok
Trying:
    sum_timestamps(['6:15:32', '2:45:48'])
Expecting:
    '9:01:20'
ok
Trying:
    sum_timestamps(['6:35:32', '2:45:48', '40:10'])
Expecting:
    '10:01:30'
ok
1 items had no tests:
    timestamps
1 items passed all tests:
   6 tests in timestamps.sum_timestamps
6 tests in 2 items.
6 passed and 0 failed.
Test passed.


TestResults(failed=0, attempted=6)

## Assignment 2: time_stamps with pytest

Write a test file to be used with pytest for assignment 1, with the same test cases.

In [67]:
%%file test_timestamps.py

from timestamps import sum_timestamps

def test_sum_time_1():
    assert sum_timestamps(['5:32', '4:48']) == '10:20'

def test_sum_time_2():
    assert sum_timestamps(['03:10', '01:00']) == '4:10'

def test_sum_time_3():
    assert sum_timestamps(['2:10', '1:59']) == '4:09'

def test_sum_time_4():
    assert sum_timestamps(['15:32', '45:48']) == '1:01:20'

def test_sum_time_5():
    assert sum_timestamps(['6:15:32', '2:45:48']) == '9:01:20'

def test_sum_time_6():
    assert sum_timestamps(['6:35:32', '2:45:48', '40:10']) == '10:01:30'


Overwriting test_timestamps.py


In [68]:
pytest.main(['-v', 'test_timestamps.py'])

============================= test session starts =============================
platform win32 -- Python 3.9.7, pytest-6.2.4, py-1.10.0, pluggy-0.13.1 -- C:\Users\klsln\anaconda3\python.exe
cachedir: .pytest_cache
rootdir: c:\Users\klsln\Documents\Python\BB1000\A2-Testing
plugins: anyio-2.2.0
collecting ... collected 6 items

test_timestamps.py::test_sum_time_1 PASSED                               [ 16%]
test_timestamps.py::test_sum_time_2 PASSED                               [ 33%]
test_timestamps.py::test_sum_time_3 PASSED                               [ 50%]
test_timestamps.py::test_sum_time_4 PASSED                               [ 66%]
test_timestamps.py::test_sum_time_5 PASSED                               [ 83%]
test_timestamps.py::test_sum_time_6 PASSED                               [100%]

============================== 6 passed in 0.03s ==============================


<ExitCode.OK: 0>

## Assignment 3: running mean

This is an example use of parametrized test cases. Look at the test function and understand how it works.
Write a function that passes the tests



In [69]:
%%file test_running_mean.py
import pytest

from running_mean import running_mean

@pytest.mark.parametrize("input_argument, expected_return", [
    ([1, 2, 3], [1, 1.5, 2]),
    ([2, 6, 10, 8, 11, 10],
     [2.0, 4.0, 6.0, 6.5, 7.4, 7.83]),
    ([3, 4, 6, 2, 1, 9, 0, 7, 5, 8],
     [3.0, 3.5, 4.33, 3.75, 3.2, 4.17, 3.57, 4.0, 4.11, 4.5]),
    ([], []),
])
def test_running_mean(input_argument, expected_return):
    ret = list(running_mean(input_argument))
    assert ret == expected_return

Overwriting test_running_mean.py


In [70]:
%%file running_mean.py
def running_mean(sequence):
    """Calculate the running mean of the sequence passed in,
       returns a sequence of same length with the averages.
       You can assume all items in sequence are numeric."""

    moving_av = []
    iter = 0
    sum = 0
   
    for i in sequence:
      iter += 1
      sum += i
      av = round(sum/iter,2)
      moving_av.append(av)
    return moving_av
    

Overwriting running_mean.py


In [71]:
pytest.main(['-v', 'test_running_mean.py'])

============================= test session starts =============================
platform win32 -- Python 3.9.7, pytest-6.2.4, py-1.10.0, pluggy-0.13.1 -- C:\Users\klsln\anaconda3\python.exe
cachedir: .pytest_cache
rootdir: c:\Users\klsln\Documents\Python\BB1000\A2-Testing
plugins: anyio-2.2.0
collecting ... collected 4 items

test_running_mean.py::test_running_mean[input_argument0-expected_return0] PASSED [ 25%]
test_running_mean.py::test_running_mean[input_argument1-expected_return1] PASSED [ 50%]
test_running_mean.py::test_running_mean[input_argument2-expected_return2] PASSED [ 75%]
test_running_mean.py::test_running_mean[input_argument3-expected_return3] PASSED [100%]

============================== 4 passed in 0.03s ==============================


<ExitCode.OK: 0>

## Assignment 4

Write a second version of the test function for timestamp where the different test cases are different parameterizations for one test function

In [72]:
%%file test_timestamps_2.py
import pytest
from timestamps import sum_timestamps

@pytest.mark.parametrize("test_input,expected", [
    (['5:32', '4:48'], '10:20'),
    (['03:10', '01:00'], '4:10'),
    (['2:10', '1:59'], '4:09'),
    (['15:32', '45:48'], '1:01:20'),
    (['6:15:32', '2:45:48'], '9:01:20'),
    (['6:35:32', '2:45:48', '40:10'], '10:01:30')])

def test_sum_time(test_input, expected):
    assert sum_timestamps(test_input) == expected


Overwriting test_timestamps_2.py


In [ ]:
pytest.main(['-v', 'test_timestamps_2.py'])

# PAST EXAM QUESTIONS

### Write a test function for the output examples in problem 12., that can be processed by pytest.

Problem 12:

    >>> def f(*args):
    ... return list(reversed(args))
    >>> f(1)
    [1]
    >>> f(1, 2)
    [2, 1]

In [ ]:
def test_f():
    assert f(1) == [1]
    assert f(1, 2) == [2, 1]